In [76]:
import pandas as pd
import numpy as np

### Working of SARIMA(p=1, d=0, q=1)(P=1, D=1, Q=0)(s=3)

| Month | Sales ($X_{t}$) |
|:-----|:---:|
| 1 | 100 |
| 2 | 110 |
| 3 | 130 |
| 4 | 105 |
| 5 | 115 |
| 6 | 135 |
| 7 | 110 |
| 8 | 120 |
| 9 | 140 |
| 10 | 115 |
| 11 | 125 |
| 12 | 145 |
| 13 | ? |

For an SARIMA(1, 0, 1)(1, 1, 0)(3) model:


​$Y_{t}$  = μ + ϕ $Y_{t-1}$ + Φ ​$Y_{t-3}$ + θ ​$ε_{t-1}$

Now, from MLE, if ϕ=0.6, Φ=0.3, θ=0.2, μ=0 the model becomes

​$Y_{t}$  = 0 + 0.6 $Y_{12}$ + 0.3 ​$Y_{10}$ + 0.2 ​$ε_{12}$

In [77]:
# --- Defining Sales data (12 records) ---
sales = [100, 110, 130, 105, 115, 135, 110, 120, 140, 115, 125, 145]
dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df = pd.DataFrame({"Sales": sales}, index=dates)

In [78]:
# --- Seasonal differencing: Y_t = X_t - X_{t-3}, with s=3 ---
s = 3
df["Y"] = df["Sales"] - df["Sales"].shift(s)
df["Y"]

1     NaN
2     NaN
3     NaN
4     5.0
5     5.0
6     5.0
7     5.0
8     5.0
9     5.0
10    5.0
11    5.0
12    5.0
Name: Y, dtype: float64

In [79]:
# --- Defining Parameters
mu = 0.0
phi = 0.6   # AR(1) coefficient
Phi = 0.3   # Seasonal AR(3) coefficient
theta = 0.2 # MA(1) coefficient

In [80]:
# --- Residual computation ---
eps = []  # will hold residuals 
for t in range(len(df)):
    if pd.isna(df["Y"].iloc[t]):
        eps.append(np.nan)   # residual undefined until we have Y_t
    else:
        Yt   = df["Y"].iloc[t]
        Y_lag1 = df["Y"].iloc[t-1] if t-1 >= 0 and not pd.isna(df["Y"].iloc[t-1]) else 0 #AR(1) 
        Y_lag3 = df["Y"].iloc[t-3] if t-3 >= 0 and not pd.isna(df["Y"].iloc[t-3]) else 0 #Seasonal AR(1) 
        eps_lag1 = eps[t-1] if t-1 >= 0 and not pd.isna(eps[t-1]) else 0 #MA(1) 
        
        # formula: ε_t = Y_t − μ − φ Y_{t−1} − Φ Y_{t−3} − θ ε_{t−1}
        res = Yt - (mu + phi*Y_lag1 + Phi*Y_lag3 + theta*eps_lag1)
        eps.append(res)

df["Residuals"] = eps

In [81]:
print(df)

    Sales    Y  Residuals
1     100  NaN        NaN
2     110  NaN        NaN
3     130  NaN        NaN
4     105  5.0   5.000000
5     115  5.0   1.000000
6     135  5.0   1.800000
7     110  5.0   0.140000
8     120  5.0   0.472000
9     140  5.0   0.405600
10    115  5.0   0.418880
11    125  5.0   0.416224
12    145  5.0   0.416755


| Month | Sales ($X_{t}$) | Seasonal Differencing $Y_{t}$ = $X_{t}$ - $X_{t-3}$ | Residulas $ε_{t}$ = $Y_{t}$ - μ - ϕ $Y_{t-1}$ - Φ ​$Y_{t-3}$ - θ ​$ε_{t-1}$ |
|:-----|:---:|:---:|-----:|
| 1 | 100 |  |  |
| 2 | 110 |  |  |
| 3 | 130 |  |  |
| 4 | 105 | 5 | 5.000000 |
| 5 | 115 | 5 | 1.000000 |
| 6 | 135 | 5 | 1.800000 |
| 7 | 110 | 5 | 0.140000 |
| 8 | 120 | 5 | 0.472000 |
| 9 | 140 | 5 | 0.405600 |
| 10 | 115 | <span style="color:blue">5</span> | 0.418888 |
| 11 | 125 | 5 | 0.416224 |
| 12 | 145 | <span style="color:red">5</span> | <span style="color:lightgreen">0.416755</span> |
| 13 | ? |  |  |

​$Y_{13}$  = 0 + 0.6 $Y_{12}$ + 0.3 ​$Y_{10}$ + 0.2 ​$ε_{12}$

​$Y_{13}$  = 0 + 0.6 (<span style="color:red">5</span>) + 0.3 (<span style="color:blue">5</span>) + 0.2 (<span style="color:lightgreen">0.416755</span>) = 4.583351

Convert back to sales:

​$X_{13}$ = ​$X_{10}$ + ​$Y_{13}$ = 115 + 4.583351 = **119.58 units**

------------------------------------------------------------------------------------------------------------------------

### Working of SARIMAX(p=1, d=0, q=1)(P=1, D=1, Q=0)(s=3)

| Month | Sales ($X_{t}$) |Ad Spent ($Z_{t}$) |
|:-----|:---:|:---:|
| 1 | 100 | 5 |
| 2 | 110 | 6 |
| 3 | 130 | 5 |
| 4 | 105 | 6 |
| 5 | 115 | 6 |
| 6 | 135 | 7 |
| 7 | 110 | 6 |
| 8 | 120 | 7 |
| 9 | 140 | 7 |
| 10 | 115 | 6 |
| 11 | 125 | 8 |
| 12 | 145 | 7 |
| 13 | ? |  |


For an SARIMAX(1, 0, 1)(1, 1, 0) model:


​$Y_{t}$  = μ + ϕ $Y_{t-1}$ + Φ ​$Y_{t-3}$ + θ ​$ε_{t-1}$ + β $Z_{t-1}$

Now, from MLE, if ϕ=0.6, Φ=0.3, θ=0.2, β=0.8, μ=0 the model becomes

​$Y_{t}$  = 0 + 0.6 $Y_{12}$ + 0.3 ​$Y_{10}$ + 0.2 ​$ε_{12}$ + 0.8 ​$Z_{12}$

In [82]:
# --- Defining sales data (12 months) ---
sales = [100, 110, 130, 105, 115, 135, 110, 120, 140, 115, 125, 145]
ad_spend = [5, 6, 5, 6, 6, 7, 6, 7, 7, 6, 8, 7]   # exogenous variable Z_t
dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

df = pd.DataFrame({
    "Sales": sales,
    "Ad_Spend": ad_spend
}, index=dates)

In [83]:
# --- Compute seasonal difference (s = 3) ---
s = 3
df["Y"] = df["Sales"] - df["Sales"].shift(s)
df["Y"]

1     NaN
2     NaN
3     NaN
4     5.0
5     5.0
6     5.0
7     5.0
8     5.0
9     5.0
10    5.0
11    5.0
12    5.0
Name: Y, dtype: float64

In [84]:
# --- Parameters (assumed) ---
mu = 0.0
phi = 0.6     # AR(1)
Phi = 0.3     # Seasonal AR(3)
theta = 0.2   # MA(1)
beta = 0.8    # Exogenous effect (Ad Spent lag 1)

In [85]:
# --- Compute residuals recursively ---
eps = []
for t in range(len(df)):
    if pd.isna(df["Y"].iloc[t]):      # first 3 rows have NaN seasonal diff
        eps.append(np.nan)
    else:
        Yt = df["Y"].iloc[t]
        Y_lag1 = df["Y"].iloc[t-1] if t-1 >= 0 and not pd.isna(df["Y"].iloc[t-1]) else 0 # AR(1)
        Y_lag3 = df["Y"].iloc[t-3] if t-3 >= 0 and not pd.isna(df["Y"].iloc[t-3]) else 0 # Seasonal AR(1)
        eps_lag1 = eps[t-1] if t-1 >= 0 and not pd.isna(eps[t-1]) else 0 # MA(1)
        Z_lag1 = df["Ad_Spend"].iloc[t-1] if t-1 >= 0 else 0 # Exogenous effect (Ad Spent lag 1)

        # --- Recursive formula for residual ---
        eps_t = Yt - (mu + phi*Y_lag1 + Phi*Y_lag3 + theta*eps_lag1 + beta*Z_lag1)
        eps.append(eps_t)

df["Residuals"] = eps

In [86]:
print(df)

    Sales  Ad_Spend    Y  Residuals
1     100         5  NaN        NaN
2     110         6  NaN        NaN
3     130         5  NaN        NaN
4     105         6  5.0   1.000000
5     115         6  5.0  -3.000000
6     135         7  5.0  -2.200000
7     110         6  5.0  -4.660000
8     120         7  5.0  -3.368000
9     140         7  5.0  -4.426400
10    115         6  5.0  -4.214720
11    125         8  5.0  -3.457056
12    145         7  5.0  -5.208589


| Month | Sales ($X_{t}$) | Ad_Spend | Seasonal Differencing $Y_{t}$ = $X_{t}$ - $X_{t-3}$ | Residulas $ε_{t}$ = $Y_{t}$ - μ - ϕ $Y_{t-1}$ - Φ ​$Y_{t-3}$ - θ ​$ε_{t-1}$ - β ​$Z_{t-1}​$|
|:-----|:---:| :---: | :---:|-----:|
| 1 | 100 | 5 |  |  |
| 2 | 110 | 6 |  |  |
| 3 | 130 | 5 |  |  |
| 4 | 105 | 6 | 5 | 1.000000 |
| 5 | 115 | 6 | 5 | -3.000000 |
| 6 | 135 | 7 | 5 |-2.200000 |
| 7 | 110 | 6 | 5 |-4.660000 |
| 8 | 120 | 7 | 5 | -3.368000 |
| 9 | 140 | 7 | 5 |-4.426400 |
| 10 | 115 | 6 | <span style="color:blue">5</span> | -4.214720 |
| 11 | 125 | 8 | 5 | -3.457056 |
| 12 | 145 | <span style="color:yellow">7</span> | <span style="color:red">5</span> | <span style="color:lightgreen">-5.208589</span> |
| 13 | ? |  |  |  |

​$Y_{13}$  = 0 + 0.6 $Y_{12}$ + 0.3 ​$Y_{10}$ + 0.2 ​$ε_{12}$ + + 0.8 ​$Z_{12}$

​$Y_{13}$  = 0 + 0.6 (<span style="color:red">5</span>) + 0.3 (<span style="color:blue">5</span>) + 0.2 (<span style="color:lightgreen">-5.208589</span>) + 0.2 (<span style="color:yellow">7</span>) = 4.86

Convert back to sales:

​$X_{13}$ = ​$X_{10}$ + ​$Y_{13}$ = 115 + 4.86 = **119.86 units**

------------------------------------------------------------------------------------------------------------------------

### Working of VAR(1)

| Month | Sales | Ad_Spent | Price |
|:-----|:---:|:---:|-----:|
| 1 | 120 | 10.0 | 50.0 |
| 2 | 130 | 11.5 | 49.8 |
| 3 | 128 | 11.0 | 50.5 |
| 4 | 140 | 12.2 | 50.2 |
| 5 | 150 | 12.8 | 50.1 |
| 6 | 145 | 12.5 | 50.3 |
| 7 | 160 | 13.2 | 50.0 |
| 8 | 170 | 14.0 | 49.9 |
| 9 | 172 | 14.5 | 50.4 |
| 10 | 180 | 15.0 | 50.2 |
| 11 | ? | ? | ? |

For computing sales equation:

In [87]:
import pandas as pd
import numpy as np

# X (9x3) - features
X = pd.DataFrame({
    'x1': [120, 130, 128, 140, 150, 145, 160, 170, 172],
    'x2': [10, 11.5, 11, 12.2, 12.8, 12.5, 13.2, 14, 14.5],
    'x3': [50, 49.8, 50.5, 50.2, 50.1, 50.3, 50, 49.9, 50.4]
})

# Y1 (Sales)   Y2 (Ad_Spent)   Y3 (Price)
Y1 = pd.DataFrame({'y1': [130, 128, 140, 150, 145, 160, 170, 172, 180]})
Y2 = pd.DataFrame({'y2': [11.5, 11, 12.2, 12.8, 12.5, 13.2, 14, 14.5, 15]})
Y3 = pd.DataFrame({'y3': [49.8, 50.5, 50.2, 50.1, 50.3, 50, 49.9, 50.4, 50.2]})

In [88]:
# Add intercept of ones
X = pd.concat([pd.Series(1, index=X.index, name='intercept'), X], axis=1)
print(X)

   intercept   x1    x2    x3
0          1  120  10.0  50.0
1          1  130  11.5  49.8
2          1  128  11.0  50.5
3          1  140  12.2  50.2
4          1  150  12.8  50.1
5          1  145  12.5  50.3
6          1  160  13.2  50.0
7          1  170  14.0  49.9
8          1  172  14.5  50.4


In [89]:
# Compute beta hat
X_mat = X.values
Y1_mat = Y1.values
beta1_hat = np.linalg.pinv(X_mat.T @ X_mat) @ X_mat.T @ Y1_mat
Y2_mat = Y2.values
beta2_hat = np.linalg.pinv(X_mat.T @ X_mat) @ X_mat.T @ Y2_mat
Y3_mat = Y3.values
beta3_hat = np.linalg.pinv(X_mat.T @ X_mat) @ X_mat.T @ Y3_mat

# Wrap into DataFrame
beta1_hat_df = pd.DataFrame(beta1_hat, index=X.columns, columns=['beta1_hat'])
print(beta1_hat_df)
beta2_hat_df = pd.DataFrame(beta2_hat, index=X.columns, columns=['beta2_hat'])
print(beta2_hat_df)
beta3_hat_df = pd.DataFrame(beta3_hat, index=X.columns, columns=['beta3_hat'])
print(beta3_hat_df)

            beta1_hat
intercept -930.503600
x1           1.637044
x2          -9.212591
x3          19.117609
           beta2_hat
intercept -74.784027
x1          0.140624
x2         -0.970757
x3          1.580826
           beta3_hat
intercept  66.819044
x1         -0.038899
x2          0.550668
x3         -0.355338


$$
X =
\begin{bmatrix}
1 & 120 & 10 & 50 \\
1 & 130 & 11.5 & 49.8 \\
1 & 128 & 11 & 50.5 \\
1 & 140 & 12.2 & 50.2 \\
1 & 150 & 12.8 & 50.1 \\
1 & 145 & 12.5 & 50.3 \\
1 & 160 & 13.2 & 50 \\
1 & 170 & 14 & 49.9 \\
1 & 172 & 14.5 & 50.4
\end{bmatrix}
$$

$$
Y_{sales} =
\begin{bmatrix}
130 \\
128 \\
140 \\
150 \\
145 \\
160 \\
170 \\
172 \\
180
\end{bmatrix}
$$


$$
\hat{β}_{sales}=
\begin{bmatrix}
{\color{red}-930.504} \\
{\color{red}1.637} \\
{\color{red}-9.213} \\
{\color{red}19.118} \\
\end{bmatrix}
$$

$$
Y_{Ad Spent} =
\begin{bmatrix}
11.5 \\
11.0 \\
12.2 \\
12.8 \\
12.5 \\
13.2 \\
14.0 \\
14.5 \\
15.0
\end{bmatrix}
$$

$$
\hat{β}_{Ad Spent} =
\begin{bmatrix}
{\color{green}-74.784} \\
{\color{green}0.141} \\
{\color{green}-0.971} \\
{\color{green}1.581} \\
\end{bmatrix}
$$

$$
Y_{price} =
\begin{bmatrix}
49.8 \\
50.5 \\
50.2 \\
50.1 \\
50.3 \\
50.0 \\
49.9 \\
50.4 \\
50.2
\end{bmatrix}
$$

$$
\hat{β}_{price} =
\begin{bmatrix}
{\color{blue}66.820} \\
{\color{blue}-0.039} \\
{\color{blue}0.551} \\
{\color{blue}-0.355}
\end{bmatrix}
$$

$$
y_t =
\begin{bmatrix}
Sales_t \\
Ad Spent_t \\
Price_t \\
\end{bmatrix}
$$

$$
c =
\begin{bmatrix}
c1 \\
c2 \\
c3 \\
\end{bmatrix}
=
\begin{bmatrix}
{\color{red}−930.504} \\
{\color{green}-74.784} \\
{\color{blue}66.820} \\
\end{bmatrix}
$$

$$
A_1 =
\begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{bmatrix}
=
\begin{bmatrix}
{\color{red}1.637} & {\color{red}−9.213} & {\color{red}19.118} \\
{\color{green}0.141} & {\color{green}−0.971} & {\color{green}1.581} \\
{\color{blue}−0.039} & {\color{blue}0.551} & {\color{blue}−0.355}
\end{bmatrix}
$$

$Sales_{t}$ = <span style="color:red">−930.504</span> + <span style="color:red">1.637</span>⋅$Sales_{t−1}$ + <span style="color:red">​− 9.213</span>⋅$AdSpend_{t−1}$ ​+ <span style="color:red">19.118</span>⋅$Price_{t−1}​$ + $ε_{1t}$

$AdSpend_t$ = <span style="color:green">−74.784</span> + <span style="color:green">0.141</span>⋅$Sales_{t−1}$ ​+ <span style="color:green">- 0.971</span>⋅$AdSpend_{t−1}$​ + <span style="color:green">1.581</span>⋅$Price_{t−1}$​ + $ε_{2t}​$

$Price_t$ = <span style="color:blue">66.820</span> + <span style="color:blue">− 0.039</span>⋅$Sales_{t−1}$ + <span style="color:blue">0.551</span>⋅$AdSpend_{t−1}$​ + <span style="color:blue">- 0.355</span>⋅$Price_{t−1}$​ + $ε_{3t}​$

Forecasting for 11th month ignoring the error terms,

$​y_{11}​$ ​ = c + $A_1​y_{10}​$ 

$Sales_{11}$ = <span style="color:red">−930.504</span> + <span style="color:red">1.637</span>(180) + <span style="color:red">- 9.213</span>(15.0) + <span style="color:red">19.118</span>(50.2) = 185.675

$AdSpend_{11}$ = <span style="color:green">−74.784</span> + <span style="color:green">0.141</span>(180) + <span style="color:green">- 0.971</span>(15.0) + <span style="color:green">1.581</span>(50.2) = 15.316

$Price_{11}$ = <span style="color:blue">66.820</span> + <span style="color:blue">− 0.039</span>(180) + <span style="color:blue">0.551</span>(15.0) + <span style="color:blue">− 0.355</span>(50.2) = 15.316

------------------------------------------------------------------------------------------------------------------------

### Working of Single Exponential Smoothing

In [90]:
# Actual sales data for 10 months
sales = [100, 25, 95, 80, 40, 45, 31, 90, 10, 23]

# Smoothing constant
alpha = 0.3

# Initialize first smoothed value
S = [sales[0]]  # S_1 = X_1

In [91]:
# Compute smoothed values for months 2–10
# Formula: S_t = α * X_(t−1) + (1 − α) * S_(t−1)
for t in range(1, len(sales)):
    S_t = alpha * sales[t - 1] + (1 - alpha) * S[t - 1]
    S.append(S_t)
    
# Print smoothed values
print("Smoothed values (S_t):")
for i in range(len(S)):
    print("Month", i + 1, "→", round(S[i], 2))

Smoothed values (S_t):
Month 1 → 100
Month 2 → 100.0
Month 3 → 77.5
Month 4 → 82.75
Month 5 → 81.92
Month 6 → 69.35
Month 7 → 62.04
Month 8 → 52.73
Month 9 → 63.91
Month 10 → 47.74


In [92]:
# Forecast next 5 months (11–15)
# For single exponential smoothing, all future forecasts = last smoothed value
forecast_value = S[-1]
forecasts = [forecast_value] * 5

# Print forecasts
print("\nForecasts for future months:")
for i in range(5):
    print("Forecast for Month", 11 + i, "→", round(forecasts[i], 2))


Forecasts for future months:
Forecast for Month 11 → 47.74
Forecast for Month 12 → 47.74
Forecast for Month 13 → 47.74
Forecast for Month 14 → 47.74
Forecast for Month 15 → 47.74


------------------------------------------------------------------------------------------------------------------------

### Working of State Space Model + Kalman Filter

In [93]:
# Computing Kalman filter update at k=1 
import numpy as np

# Given matrices from the conversation
A = np.array([[0.5, -4.0],
              [0.1, -0.05]])
B = np.array([[0.3],
              [0.0]])
C = np.array([[0.2, 0.0],
              [0.05, 0.0],
              [0.0, 0.25],
              [0.0, 0.10]])
D = np.zeros((4,2))

x0 = np.array([[100.0],
               [10.0]])
P0 = np.array([[10.0, 0.0],
               [0.0, 10.0]])
Q = np.array([[5.0, 0.0],
              [0.0, 2.0]])
R = np.array([[4.0,0,0,0],
              [0,4.0,0,0],
              [0,0,1.0,0],
              [0,0,0,1.0]])
u1 = np.array([[1.0]])

# Observed value
y1 = np.array([[21.72],
               [5.35],
               [5.76],
               [4.16]])

In [94]:
# Predict
x_pred = A.dot(x0) + B.dot(u1)
P_pred = A.dot(P0).dot(A.T) + Q

# Innovation
innovation = y1 - C.dot(x_pred)

# Innovation covariance
S = C.dot(P_pred).dot(C.T) + R

print("Predicted state x_{1|0}:\n", x_pred)
print("\nPredicted covariance P_{1|0}:\n", P_pred)

Predicted state x_{1|0}:
 [[10.3]
 [ 9.5]]

Predicted covariance P_{1|0}:
 [[167.5     2.5  ]
 [  2.5     2.125]]


In [95]:
# Kalman gain
K = P_pred.dot(C.T).dot(np.linalg.inv(S))

# Posterior
x_post = x_pred + K.dot(innovation)
P_post = (np.eye(2) - K.dot(C)).dot(P_pred)

print("\nKalman gain K:\n", K)
print("\nPosterior(hidden) state x_1:\n", x_post)
print("\nPosterior covariance P_1:\n", P_post)


Kalman gain K:
 [[3.01038401 0.752596   0.19512234 0.07804894]
 [0.03902447 0.00975612 0.45583932 0.18233573]]

Posterior(hidden) state x_1:
 [[74.0339776 ]
 [12.44270563]]

Posterior covariance P_1:
 [[60.20768029  0.78048935]
 [ 0.78048935  1.82335727]]
